<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [2]:


# YOUR CODE HERE
from pathlib import Path
import zipfile
import requests
import os

data_path= Path("data")
images_path = data_path /"pizza_steak_sushi"

images_path.mkdir(parents=True, exist_ok=True)


with open(data_path/"pizza_steak_sushi.zip","wb") as f:
    requete= requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    f.write(requete.content)

with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip","r") as f:
    f.extractall(images_path)

os.remove(data_path/"pizza_steak_sushi.zip")

In [ ]:
# Example running of get_data.py
!python get_data.py

## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [14]:
%%writefile make_dataloader.py

###Préparation des datasets
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch

train_path=images_path / "train"
test_path=images_path/ "test"


def make_dataloader(train_path,test_path,transformer,batch_size):


    trainset = datasets.ImageFolder(root=train_path,
                                    transform=transformer)
    
    testset = datasets.ImageFolder(root=test_path,
                                    transform=transformer)
    

    train_dl=DataLoader(dataset=trainset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=os.cpu_count())
    
    test_dl=DataLoader(dataset=testset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=os.cpu_count())
    
    return train_dl, test_dl

from torch import nn
import torch.utils.data

Writing make_dataloader.py


In [11]:
%%writefile train_modele.py

def train_step(model:nn.Module,loss_fn:nn.Module,optimizer:torch.optim,device,dataloader:torch.utils.data.DataLoader):

    model.train()
    
    train_loss = 0
    train_acc = 0


    for batch, (x,y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)

        pred = model(x)

        loss=loss_fn(pred,y)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        pred_label = torch.argmax(torch.softmax(pred,dim=1),dim=1)
        train_acc += (pred_label == y).sum().item()/len(pred)

    train_loss = train_loss /len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc


def test_step(model:nn.Module,loss_fn,device,dataloader):

    model.eval()
    test_loss=0
    test_acc=0

    with torch.inference_mode():
        for batch, (x,y) in enumerate(dataloader):
            x = x.to(device)
            y = y.to(device)

            pred = model(x)

            loss = loss_fn(pred,y)
            test_loss += loss.item()
            
            label = torch.argmax(torch.softmax(pred,dim=1),dim=1)
            test_acc += (label == y).sum().item()/len(pred)

    test_loss = test_loss/len(dataloader)
    test_acc = test_acc/len(dataloader)

    return test_loss, test_acc


def train(model,train_dataloader,test_dataloader,device,optimizer,loss_fn,epochs):

    from tqdm.auto import tqdm
    for epoch in tqdm(range(epochs)):

        results={"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}

        train_loss,train_acc=train_step(model=model,
                                        dataloader=train_dataloader,
                                        device=device,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
        
        test_loss, test_acc=test_step(model=model,
                                      loss_fn=loss_fn,
                                      device=device,
                                      dataloader=test_dataloader,
                                      )
        

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


    return results

Writing train_modele.py


In [10]:
%%writefile tinyvgg.py

import torch
from torch import nn 

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/
    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()

        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion


Writing tinyvgg.py


In [9]:
%%writefile savefile.py

def save_model(model:nn.Module,nom_modele):

    models_path=Path("D:\Adam\Pytorch\exercises")
    nom_du_modele = nom_modele.pt

    model_path=models_path/nom_du_modele

    torch.save(obj=model.state_dict(),f=model_path)

Writing savefile.py


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [ ]:
# YOUR CODE HERE

In [ ]:
# Example running of predict.py 
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg